In [34]:
import sqlite3
import pandas as pd
from glob import glob
from os.path import join

In [31]:
conn = sqlite3.connect(join('Dota_data','mixed_skill','dota_data.db'))

In [135]:
conn.close()

In [35]:
path = glob(join('Dota_data','mixed_skill','*.csv'))
print(path[1])
dataDF = pd.read_csv(path[1])
print(list(dataDF))
del dataDF['Unnamed: 0'], dataDF['dire_tower_status'], dataDF['radiant_tower_status']
print(list(dataDF))

Dota_data/mixed_skill/20191111API_4725403110-4781085451.csv
['Unnamed: 0', 'match_id', 'start_time', 'duration', 'game_mode', 'lobby_type', 'radiant_win', 'radiant_kills', 'dire_kills', 'radiant_gold', 'dire_gold', 'radiant_xp', 'dire_xp', 'radiant_tower_status', 'dire_tower_status', 'first_blood_time', 'radiant_hero0', 'dire_hero0', 'radiant_hero1', 'dire_hero1', 'radiant_hero2', 'dire_hero2', 'radiant_hero3', 'dire_hero3', 'radiant_hero4', 'dire_hero4', 'skill_level']
['match_id', 'start_time', 'duration', 'game_mode', 'lobby_type', 'radiant_win', 'radiant_kills', 'dire_kills', 'radiant_gold', 'dire_gold', 'radiant_xp', 'dire_xp', 'first_blood_time', 'radiant_hero0', 'dire_hero0', 'radiant_hero1', 'dire_hero1', 'radiant_hero2', 'dire_hero2', 'radiant_hero3', 'dire_hero3', 'radiant_hero4', 'dire_hero4', 'skill_level']


In [27]:
dataDF.to_sql('dota_data', con=conn, if_exists='replace')

In [43]:
conn.enable_load_extension(True)
conn.load_extension('libsqlitefunctions')

In [128]:
c.execute('DROP TABLE dota_stats')
c.execute('CREATE TABLE dota_stats(condition,col_name,avg,stdev,max,min)')

In [129]:
c = conn.cursor()

#get stats of various params in games where radiant won
#  and put them into dota_stats database
#Use list(df) to get column names, but if not in SQLite 
#  it would be easy to do this from schema?
for column in list(dataDF)[:-11]:
    #if column=='match_id' or column=='game_mode' or column=='lobby_type':
    #    continue
    c.execute("INSERT INTO dota_stats(condition,col_name,avg,stdev,max,min)\
                SELECT 'radiant_win', ? , \
                AVG({cn}), STDEV({cn}), \
                MAX({cn}), MIN({cn}) \
                FROM dota_data WHERE radiant_win=1".format(cn=column),
                [column])
c.fetchall()

[]

In [130]:
dota_stats_all = c.execute('SELECT * FROM dota_stats').fetchall()

In [131]:
len(dota_stats_all)

13

In [132]:
for row in dota_stats_all:
    print(row)

('radiant_win', 'match_id', 4757139026.7257805, 19890617.820782714, 4781085451, 4725403110)
('radiant_win', 'start_time', 1558033148.6386366, 579093.2340484913, 1558752238, 1557129842)
('radiant_win', 'duration', 1838.5001047559188, 687.1643518040914, 7123, 0)
('radiant_win', 'game_mode', 20.41162727209239, 5.627929082668111, 23, 2)
('radiant_win', 'lobby_type', 3.5590410706689775, 3.4995130952277393, 7, 0)
('radiant_win', 'radiant_win', 1.0, 0.0, 1, 1)
('radiant_win', 'radiant_kills', 37.047717908183024, 11.63068060090767, 115, 0)
('radiant_win', 'dire_kills', 23.48315334361846, 12.47266827978516, 99, 0)
('radiant_win', 'radiant_gold', 90035.30421901876, 30963.60278861686, 444829.06666666665, 0.0)
('radiant_win', 'dire_gold', 69107.51261113054, 33367.627254950035, 415574.3999999999, 0.0)
('radiant_win', 'radiant_xp', 114937.9343375082, 48830.00261155748, 542318.2666666667, 0.0)
('radiant_win', 'dire_xp', 94888.73379457762, 51958.225575698125, 507291.73333333345, 0.0)
('radiant_win', '

In [134]:
c.execute("SELECT sql FROM sqlite_master \
            WHERE tbl_name = 'dota_data' AND type = 'table'")
c.fetchall()

[('CREATE TABLE "dota_data" (\n"index" INTEGER,\n  "match_id" INTEGER,\n  "start_time" INTEGER,\n  "duration" INTEGER,\n  "game_mode" INTEGER,\n  "lobby_type" INTEGER,\n  "radiant_win" INTEGER,\n  "radiant_kills" INTEGER,\n  "dire_kills" INTEGER,\n  "radiant_gold" REAL,\n  "dire_gold" REAL,\n  "radiant_xp" REAL,\n  "dire_xp" REAL,\n  "first_blood_time" INTEGER,\n  "radiant_hero0" INTEGER,\n  "dire_hero0" INTEGER,\n  "radiant_hero1" INTEGER,\n  "dire_hero1" INTEGER,\n  "radiant_hero2" INTEGER,\n  "dire_hero2" INTEGER,\n  "radiant_hero3" INTEGER,\n  "dire_hero3" INTEGER,\n  "radiant_hero4" INTEGER,\n  "dire_hero4" INTEGER,\n  "skill_level" TEXT\n)',)]

In [45]:
c.execute("SELECT MAX(radiant_kills),AVG(radiant_kills),\
            STDEV(radiant_kills),MAX(dire_kills), \
            AVG(dire_kills), STDEV(dire_kills), \
            COUNT(*) FROM dota_data WHERE radiant_win = 0").fetchall()

[(91,
  24.59660774889049,
  12.105078932631068,
  167,
  39.16982754649181,
  11.63748604494706,
  130238)]